In [1]:
import pandas as pd
import numpy as np

In [7]:
#Parses the LibFM output model, returns w0, wj, and vj,f.
#w0 is a single number, wj is an array of floats, vj,f is an array of array of floats
def parse_output_file(filename):
    w0_spot=False
    wj_spot=False
    vj_spot=False
    w0=0
    wj=[]
    vj=[]
    f = open(filename, 'r')
    for line in f:
        if 'global bias' in line:
            w0_spot=True
        elif w0_spot:
            w0=float(line)
            w0_spot=False
        elif 'unary interactions' in line:
            wj_spot=True
        elif 'pairwise interactions' in line:
            wj_spot=False
            vj_spot=True
        elif wj_spot:
            wj.append(float(line))
        elif vj_spot:
            line_split=line.split(' ')
            line_numbers = [float(i) for i in line_split]
            vj.append(line_numbers)
    return w0,wj,vj

In [9]:
w0, wj, vj = parse_output_file('model1')

In [32]:
#THIS NEEDS TO BE VECTORIZED

def make_prediction(input_vector, w0, wj, vj):
    N=len(input_vector)
    unary_sum=np.dot(input_vector,wj)
    binary_sum=0
    for x in range(N):
        for y in range(x+1, N):
            dot = np.dot(vj[x],vj[y])
            dot_val = dot*input_vector[x]*input_vector[y]
            binary_sum+=dot_val
    return w0+unary_sum+binary_sum

In [17]:
num_col=len(wj)

In [33]:
#Given a string (line from the libfm testing set), outputs an input vector
def create_input_vector(sample, num_col): 
    input_vector=[0]*num_col
    sample_split = sample.split(' ')
    sample_split = sample_split[1:]
    for s in sample_split:
        col_val = int(s[:s.index(':')])
        item_val = float(s[s.index(':')+1:])
        input_vector[col_val]=item_val
    return input_vector


KeyboardInterrupt: 

In [ ]:
sample = '3 10714:1 60589:1 63429:0.006412124744972311 63437:0.01399009035266686 63438:0.3672398717575051 63439:0.1002623141941125 63442:0.08394054211600116 63444:0.004954823666569514 63449:0.06324686680268143 63451:0.1839113960944331 63456:0.1305741766248907 63458:0.03410084523462548 63465:0.01136694841154182'
input_vector = create_input_vector(sample, num_col)
make_prediction(input_vector,w0,wj,vj)